# Libraries

In [ ]:
# import os
import json
import pandas as pd
from openai import OpenAI
from datasets import load_dataset
from sklearn.metrics import f1_score
from pprint import pprint

pd.set_option('max_colwidth', None)

# Config

In [ ]:
seed = 42

client = OpenAI(api_key=json.load(open('secrets.json'))['OPENAPI_SECRET_KEY'])

idn2eng_emotion_map = {
    'marah': 'anger',
    'jijik': 'disgust',
    'takut': 'fear',
    'senang': 'joy',
    'sedih': 'sadness',
    'terkejut': 'surprise',
    'biasa': 'neutral',
}

# Data

## Load Data

In [ ]:
df = pd.read_csv('data/preprocessed_data/track_a/sun_70_15_15_stratify_v2/val.csv')
df = df.rename(columns=idn2eng_emotion_map)
emotion_cols = [col for col in df.columns if col not in ['Unnamed: 0', 'text', 'emotion', 'id']]
df['emotion'] = df.apply(lambda row: ', '.join([emotion for emotion in emotion_cols if row[emotion] == 1]), axis=1)

print("DF size:", len(df))
print("Emotion columns:", emotion_cols)
df.head()

In [ ]:
# Omit data with 'emotion' "neutral"
# df = df[~(df['emotion'] == 'neutral')]
# print("DF size (after removing emotion 'neutral'):", len(df))

# Evaluation

In [ ]:
# prompt = """# INSTRUCTION
# Identify the EMOTION(S) expressed in the TEXT. 
# This is a multi-label classification task, meaning the identified emotion(s) may include one or more labels from the predefined EMOTION LABELS below.

# ## EMOTION LABELS 
# - anger
# - fear
# - joy
# - sadness
# - surprise

# ## STEPS TO IDENTIFY THE EMOTION(S)
# 1. Extract all KEYWORD(S) that carry significant emotion-related information in the TEXT. For each `keyword`, specify the `related emotion` from the EMOTION LABELS in the format: `keyword -> related emotion`.
# 2. Based on the identified KEYWORD(S), determine the overall EMOTION(S) expressed in the TEXT.

# The OUTPUTS must consist of KEYWORD(S) and EMOTION(S).
# Only respond the KEYWORD(S) and EMOTION(S).

# # INPUT
# ## TEXT
# {text}

# # OUTPUTS
# ## KEYWORD(S)
# """

# ## TIPS
# - If the TEXT contains laughter-like expressions such as "hahaha," "wkwkw," or similar, the identified emotion(s) must at least include "joy."
# - If the TEXT contains variants of smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, the identified emotion(s) must at least include "joy."

# prompt = """# INSTRUCTION
# Identify the EMOTION(S) expressed in the TEXT. 
# This is a multi-label classification task, so there will always be one or more emotions to identify in the provided TEXT. 
# The model must output the KEYWORD(S) and the EMOTION(S) from the predefined EMOTION LABELS below. 
# The model must always return a result—there should never be an instance with no identified emotion(s).

# ## EMOTION LABELS 
# - anger
# - fear
# - joy
# - sadness
# - surprise

# ## STEPS TO IDENTIFY THE EMOTION(S)
# 1. Extract all KEYWORD(S) that carry significant emotion-related information in the TEXT. For each `keyword`, specify the `related emotion` from the EMOTION LABELS in the format: `keyword -> related emotion (reason)`.
# 2. Based on the identified KEYWORD(S), determine the overall EMOTION(S) expressed in the TEXT.

# ## TIPS
# - If the TEXT contains laughter-like expressions such as "hahaha," "wkwkw," or similar, the identified emotion(s) often include "joy."
# - If the TEXT contains variants of smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, the identified emotion(s) usually include "joy."
# - If the TEXT contains insults, the identified emotion(s) usually include both "disgust" and "anger."
# - In Sundanese, "NU" is commonly used to mean "yang" (in Indonesian), not "Nahdlatul Ulama." The model must interpret "NU" accordingly based on the context.

# The OUTPUTS must include the KEYWORD(S) and the EMOTION(S).
# Do not omit any emotion expressed in the TEXT.

# # INPUT
# ## TEXT
# {text}

# # OUTPUTS
# ## KEYWORD(S)
# """

# prompt = """# INSTRUCTION
# Identify the EMOTION(S) expressed in the TEXT. 
# This is a multi-label classification task, so there will always be one or more emotions to identify in the provided TEXT. 
# The model must output the KEYWORD(S) and the EMOTION(S) from the predefined EMOTION LABELS below. 
# The model must always return a result—there should never be an instance with no identified emotion(s).

# ## EMOTION LABELS 
# - anger
# - fear
# - joy
# - sadness
# - surprise

# ## STEPS TO IDENTIFY THE EMOTION(S)
# 1. Extract all KEYWORD(S) that carry significant emotion-related information in the TEXT. For each `keyword`, specify the `related emotion` from the EMOTION LABELS in the format: `keyword -> related emotion (reason)`.
# 2. Based on the identified KEYWORD(S), determine the overall EMOTION(S) expressed in the TEXT.

# ## TIPS
# - If the TEXT contains laughter-like expressions such as "hahaha," "wkwkw," or similar, the identified emotion(s) often include "joy."
# - If the TEXT contains variants of smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, the identified emotion(s) usually include "joy."
# - If the TEXT contains insults, the identified emotion(s) usually include both "disgust" and "anger."
# - In Sundanese, "NU" is commonly used to mean "yang" (in Indonesian), not "Nahdlatul Ulama." The model must interpret "NU" accordingly based on the context.
# - The emoji 😭 does not always indicate "sadness". It can sometimes represent extreme laughter that brings tears. In such cases, the identified emotion(s) should include "joy."

# The OUTPUTS must include the KEYWORD(S) and the EMOTION(S).
# Do not omit any emotion expressed in the TEXT.

# # INPUT
# ## TEXT
# {text}

# # OUTPUTS
# ## KEYWORD(S)
# """

# prompt = """# INSTRUCTION
# Identify the EMOTION(S) expressed in the TEXT. 
# This is a multi-label classification task, so there will always be one or more emotions to identify in the provided TEXT. 
# The model must output the KEYWORD(S) and the EMOTION(S) from the predefined EMOTION LABELS below. 
# The model must always return a result—there should never be an instance with no identified emotion(s).

# ## EMOTION LABELS 
# - anger
# - fear
# - joy
# - sadness
# - surprise

# ## STEPS TO IDENTIFY THE EMOTION(S)
# 1. Extract all KEYWORD(S) that carry significant emotion-related information in the TEXT. For each `keyword`, specify the `related emotion` from the EMOTION LABELS in the format: `keyword -> related emotion (reason)`.
# 2. Based on the identified KEYWORD(S), determine the overall EMOTION(S) expressed in the TEXT.

# ## TIPS
# - If the TEXT contains laughter-like expressions such as "hahaha," "wkwkw," or similar, the identified emotion(s) often include "joy."
# - If the TEXT contains variants of smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, the identified emotion(s) usually include "joy."
# - If the TEXT contains insults, the identified emotion(s) usually include both "disgust" and "anger."
# - In Sundanese, "NU" is commonly used to mean "yang" (in Indonesian), not "Nahdlatul Ulama." The model must interpret "NU" accordingly based on the context.
# - The emoji 😭 does not always indicate "sadness". It can sometimes represent extreme laughter that brings tears. In such cases, the identified emotion(s) should include "joy."

# The OUTPUTS must include the KEYWORD(S) and the EMOTION(S).
# Do not omit any emotion expressed in the TEXT.

# # INPUT
# ## TEXT
# {text}

# # OUTPUTS
# ## KEYWORD(S)
# """

# F1 Macro: 0.5238851095993953
prompt = """# INSTRUCTION
Identify the EMOTION(S) expressed in the TEXT. 
This is a multi-label classification task, meaning there will always be one or more emotions to identify in the TEXT. 
The model must output the EMOTION(S) from the predefined EMOTION LABELS listed below. 
The model must always return a result—there should never be an instance with no identified EMOTION(s).

## EMOTION LABELS
```
- anger
- fear
- joy
- sadness
- surprise
```

## STEPS TO IDENTIFY THE EMOTION(S)
1. Guess the CONTEXT of the TEXT by analyzing its overall meaning and tone. Provide a concise description of the CONTEXT to clarify the emotional environment in which the TEXT is situated.
2. Using the guessed CONTEXT, Extract all KEYWORD(S) that carry significant emotion-related information in the TEXT. For each `keyword`, specify the `related emotion` from the EMOTION LABELS in the format: `keyword -> related emotion (reason)`. Ensure that the extracted KEYWORD(S) align with the described CONTEXT to maintain consistency in interpretation.
3. Based on the identified KEYWORD(S), determine the overall EMOTION(S) expressed in the TEXT.

## TIPS
- Laughter-like expressions, such as "hahaha," "wkwkw," or similar, typically indicate `joy`.
- Smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, typically indicate `joy`.
- Insults typically indicate `anger` and may also suggest `disgust`.
- The emoji 😭 does not always indicate `sadness`. In humorous contexts, it can represent extreme laughter that brings tears, indicating `joy`.
- In Sundanese, "NU" often means "yang" in Indonesian, not "Nahdlatul Ulama". Always interpret "NU" based on the context.

The OUTPUTS must include the CONTEXT, KEYWORD(S) and EMOTION(S).
Do not omit any emotion expressed in the TEXT.

# INPUT
## TEXT
{text}

# OUTPUTS
## CONTEXT
"""

In [ ]:
def predict(text):
    completion = client.chat.completions.create(
        model="gpt-4o",
        # model="gpt-4o-mini",
        messages=[
            # {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                'content': prompt.format(text=text),
            },
        ],
        max_tokens=300,
    )
    return completion.choices[0].message.content

def one_hot_encode_emotions(emotions, emotion_cols):
    one_hot_emotion = [1 if emotion_col in emotions else 0 for emotion_col in emotion_cols]
    return one_hot_emotion

In [ ]:
# df_sampled = df.sample(n=20)
# y_true = df_sampled[emotion_cols].to_numpy()

# y_pred = []
# for i, v in enumerate(df_sampled.iterrows()):
#     idx, row = v

#     print("=" * 64)
#     print(f"{i+1} of {len(df_sampled)}")
#     print("=" * 64)

#     is_valid = True

#     while True:
#         print("Predicting...")        

#         output = predict(row['text'])
#         ctx = prompt + output

#         keywords_emotions = ctx.split("# KEYWORD(S)")[-1].split("## EMOTION(S)")
#         # keywords = keywords_emotions[0].strip().split("\n")
#         keywords = keywords_emotions[0].strip()
#         emotions = [emotion.replace("-", "").strip() for emotion in keywords_emotions[-1].strip().split("\n")]

#         is_valid = bool(sum([int(emotion_pred in emotion_cols) for emotion_pred in emotions]))

#         if is_valid:
#             break
#         else:
#             print("ERROR:", output)
#             print()
        
#     emotions_one_hot = one_hot_encode_emotions(emotions, emotion_cols)
#     y_pred.append(emotions_one_hot)

#     print()
#     print("# TEXT")
#     print(row['text'])
#     print()
#     print("# KEYWORD(S)")
#     print(keywords)
#     print()
#     print("# PREDICTED EMOTION(S)")
#     print(emotions)
#     print()
#     print("# TRUE EMOTION(S)")
#     print(row['emotion'])
#     print()
#     # df.at[i, 'text_sun2idn'] = text_sun2idn

# f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=1.0)
# print("F1 Macro:", f1_macro)

In [ ]:
# df_sampled = df.sample(n=20)
y_true = df[emotion_cols].to_numpy()

y_pred = []
for i, v in enumerate(df.iterrows()):
    idx, row = v

    print("=" * 64)
    print(f"{i+1} of {len(df)}")
    print("=" * 64)

    is_valid = True

    while True:
        print("Predicting...")        

        output = predict(row['text'])
        prompt_output = prompt + output

        outputs = prompt_output.split("# OUTPUTS")[-1]
        ctx = outputs.split("## CONTEXT")[-1].split("## KEYWORD(S)")[0].strip()
        keywords = outputs.split("## KEYWORD(S)")[-1].split("## EMOTION(S)")[0].strip()
        emotions = outputs.split("## EMOTION(S)")[-1].strip()
        emotions = [emotion.replace("-", "").strip() for emotion in emotions.split("\n")]

        is_valid = bool(sum([int(emotion_pred in emotion_cols) for emotion_pred in emotions]))

        if is_valid:
            break
        else:
            print("ERROR:", output)
            print()
        
    emotions_one_hot = one_hot_encode_emotions(emotions, emotion_cols)
    y_pred.append(emotions_one_hot)

    print()
    print("# TEXT")
    print(row['text'])
    print()
    print("# CONTEXT")
    print(ctx)
    print()
    print("# KEYWORD(S)")
    print(keywords)
    print()
    print("# PREDICTED EMOTION(S)")
    print(emotions)
    print()
    print("# TRUE EMOTION(S)")
    print(row['emotion'])
    print()
    # df.at[i, 'text_sun2idn'] = text_sun2idn

f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=1.0)
print("F1 Macro:", f1_macro)

## Evaluation - Prompt #2

In [ ]:
# prompt = """# INSTRUCTION
# Identify the EMOTION(S) expressed in the TEXT. 
# This is a multi-label classification task, meaning there will always be one or more emotions to identify in the TEXT. 
# The model must output the EMOTION(S) from the predefined EMOTION LABELS listed below. 
# The model must always return a result—there should never be an instance with no identified EMOTION(s).

# ## EMOTION LABELS
# ```
# - anger
# - fear
# - joy
# - sadness
# - surprise
# ```

# ## STEPS TO IDENTIFY THE EMOTION(S)
# 1. Guess the CONTEXT of the TEXT by analyzing its overall meaning and tone. Provide a concise description of the CONTEXT to clarify the emotional environment in which the TEXT is situated.
# 2. Using the guessed CONTEXT, Extract all KEYWORD(S) that carry significant emotion-related information in the TEXT. For each `keyword`, specify the `related emotion` from the EMOTION LABELS in the format: `keyword -> related emotion (reason)`. Ensure that the extracted KEYWORD(S) align with the described CONTEXT to maintain consistency in interpretation.
# 3. Based on the identified KEYWORD(S), determine the overall EMOTION(S) expressed in the TEXT.

# ## TIPS
# - Laughter-like expressions, such as "hahaha," "wkwkw," or similar, typically indicate `joy`.
# - Smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, typically indicate `joy`.
# - Insults typically indicate `anger` and may also suggest `disgust`.
# - The emoji 😭 does not always indicate `sadness`. In humorous contexts, it can represent extreme laughter that brings tears, indicating `joy`.
# - In Sundanese, "NU" often means "yang" in Indonesian, not "Nahdlatul Ulama". Always interpret "NU" based on the context.

# The OUTPUTS must include the CONTEXT, KEYWORD(S) and EMOTION(S).
# Do not omit any emotion expressed in the TEXT.

# # INPUT
# ## TEXT
# {text}

# # OUTPUTS
# ## CONTEXT
# """

prompt = (
"You will be provided with a text in Sundanese. "
"Your task is to identify the emotion(s) expressed in the text. "
"This is a multi-label classification task, meaning the text may contain one or more emotions. "
"""You must output the identified emotion(s) from the following predefined labels:
- anger
- fear
- joy
- sadness
- surprise
"""
"You must always provide a result; there should never be an instance with no identified emotion(s)."
"\n"
"""
Use the following step-by-step instructions to identify the emotion(s):
1. Extract all keyword(s) related to the writer's situation or context when the text was written. These keyword(s) should help infer what might have been happening to the writer or around them. For example, look for mentions of events, objects, or specific phrases that suggest a situation.
2. Based on the extracted situation-related keyword(s), make an assumption about the writer's situation or context. 
3. Extract all keyword(s) that carry significant emotion-related information in the text. For each keyword, specify the emotion using the predefined labels in the format: keyword -> emotion (reason).
4. Based on the identified keyword(s), determine the overall emotion(s) expressed in the text.
"""
"""
Consider the following additional context when specifying the emotion(s) of the identified keyword(s):
- Laughter-like expressions, such as "hahaha," "wkwkw," or similar, typically indicate joy.
- Smile or laugh emojis, such as 😊, ☺️, 😃, 😁, 😆, 😇, 🥰, 😍, 😋, 🤗, 😌, typically indicate joy.
- Insults typically indicate anger and may also suggest disgust.
- The emoji 😭 does not always indicate sadness. In humorous contexts, it can represent extreme laughter that brings tears, indicating joy.
- In Sundanese, "NU" often means "yang" in Indonesian, not "Nahdlatul Ulama". Always interpret "NU" based on the context.
"""
"""
Your output must follow this exact structure:
<situation_keywords>
A markdown list of one or more situation-related keywords.
</situation_keywords>

<situation_assumption>
The detailed assumption about the writer’s situation when they wrote the text.
</situation_assumption>

<emotion_keywords>
A markdown list of one or more emotion-related keywords in the format: keyword -> emotion (reason).
</emotion_keywords>

<emotions>
A list of one or more emotions separated by commas.
</emotions>
"""
)

print(prompt)

In [ ]:
def predict(text):
    completion = client.chat.completions.create(
        model='gpt-4o',
        # model='gpt-4o-mini',
        messages=[
            {'role': 'system', 'content': prompt},
            {
                'role': 'user',
                'content': text,
            },
        ],
        max_tokens=1000,
    )
    return completion.choices[0].message.content

output = predict("Teu cukup sakali ngadangu keun na duh , tuluuuuunggg ,")
print(output)

## Save Evaluation

In [ ]:
save_df = df.copy()
save_df[emotion_cols] = y_pred
save_df.to_csv('eval_sun.csv', index=False)
print("Saved to: eval_sun.csv")